In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

The following is a naive simulation of fish movements.

Assumptions:

1. Number of fish in poisson distributed with expected count of 1000 fish.
2. Fish move in a general direction with a speed of 50 meters per minute with
    some random gaussian noise with std = 1 meter.
3. Detection radius of each sonar sensor is 150m.
4. Distribution of fish in the school is a multivariate normal distribution.

In [2]:
# set sensor positions
sensors = np.array([
    np.array([[i]*4 for i in range(4)]).flatten(),
    np.array(list(range(4))*4).flatten()
]).T*250

In [3]:
# assume multivariate distribution of fish in school
mean = np.array([0, 0])
cov = np.array([[50, 5],
                [5, 20]])

n_fishes = np.random.poisson(1000)
position = np.random.multivariate_normal(mean, cov, n_fishes)

# for this simulation, we use a linear path with some small random noise
def path_linear(p, vx=20, vy=40, sig=1):
    # position after 5 mins
    new_x = p[:, 0] + (5*vx + np.random.normal(scale=sig))
    new_y = p[:, 1] + (5*vy + np.random.normal(scale=sig))
    return np.array([new_x, new_y]).T

# generate positions of fish every 5 mins (total of 288 instances in a day)
positions = [position]
for i in range(1, 288):
    positions.append(path_linear(positions[i-1]))

In [4]:
def count_fish(positions, sensors=sensors):
    fish_counts = [[(np.linalg.norm(p - sensors[i, :], axis=1) < 150).sum()
                    for i in range(16)]
                   for p in positions]
    df = pd.DataFrame(fish_counts, columns=[f'sensor{i}' for i in range(16)])
    return df

In [5]:
df_positions = count_fish(positions)

In [6]:
# plot fish simulations and store as png files

# for t in range(len(df_positions)):
#     p = positions[t]
#     fig, ax = plt.subplots(1, figsize=(8, 8))
#     ax.plot(p[:, 0], p[:, 1], 'r.')
#     ax.set_xlim(-150, 900)
#     ax.set_ylim(-150, 900)

#     for i in range(4):
#         for j in range(4):
#             ax.add_artist(plt.Circle((250*i, 250*j),
#                                      radius=150, color='blue', alpha=0.1))
#     ax.set_title(f'Fish Detected at t={t}')
#     ax.set_xticks(np.array(range(4))*250)
#     ax.set_yticks(np.array(range(4))*250)
#     ax.grid(True)
#     count = f'{t}'.zfill(3)
#     plt.savefig(f'./fishes/fish_{count}')
#     plt.close();

In [7]:
from IPython.display import Image
Image(url='./fishes/fish_detector_100.gif')